In [1]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib oauth2client

In [2]:
# GA
# https://stackoverflow.com/questions/12837748/analytics-google-api-error-403-user-does-not-have-any-google-analytics-account
# https://stackoverflow.com/questions/59451776/parsing-google-api-object-into-pandas-dataframe
# https://developers.google.com/analytics/devguides/reporting/core/v4
#
# GSC
# https://developers.google.com/search/apis/indexing-api/v3/using-api?hl=tr
# https://developers.google.com/search/apis/indexing-api/v3/prereqs?hl=tr
# https://developers.google.com/webmaster-tools/search-console-api-original/v3/quickstart/quickstart-python

In [3]:
#import os.path
import pandas as pd
#import json
#import httplib2

#from apiclient.discovery import build
#from oauth2client.service_account import ServiceAccountCredentials
#from apiclient import errors
#from apiclient.discovery import build
#from oauth2client.client import OAuth2WebServerFlow

from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
# GENERIC
KEY_FILE_LOCATION = 'r-test-308919-e4b2af358d7e.json'
DATE_RANGES = [{
    'startDate': '2020-06-30',
    'endDate': '2021-06-30'
}]
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly', 'https://www.googleapis.com/auth/webmasters.readonly']
CREDENTIALS = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)

# GA
GA_VIEW_ID = '109466096'
GA_BODY = {
        'reportRequests': [{
            'viewId': GA_VIEW_ID,
            'dateRanges': DATE_RANGES,
            'metrics': [
                # {
                #     'expression': 'ga:sessions',
                #     'alias': 'Sessions'
                # },
                {
                    'expression': 'ga:bounceRate',
                    'alias': 'BounceRate'
                },
                {
                    'expression': 'ga:pageviews',
                    'alias': 'Pageviews'
                },
                {
                    'expression': 'ga:avgTimeOnPage',
                    'alias': 'AvgTimeOnPage'
                },
                {
                    'expression': 'ga:avgSessionDuration',
                    'alias': 'AvgSessionDuration'
                },
                {
                    'expression': 'ga:avgPageLoadTime',
                    'alias': 'AvgPageLoadTime'
                },
            ],
            'dimensions': [
                {'name': 'ga:pagePath'},
                {'name': 'ga:pageTitle'},
            ],
            'orderBys': [{
                "fieldName": "ga:pageviews",
                "orderType": "VALUE",
                "sortOrder": "DESCENDING"
            }],
            #"segments":[{
            #    "segmentId": "gaid::-3"
            #}],     "pageSize": 10000,
        }],
    }

# GSC
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'
URL = 'https://ceaksan.com'
GSC_BODY = {
    'startDate': DATE_RANGES[0]['startDate'],
    'endDate': DATE_RANGES[0]['endDate'],
    'dimensions': ['query', 'page'],
    'rowLimit': 10000
}

In [5]:
def print_response(response):
    list = []

    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for row in report.get('data', {}).get('rows', []):
        dict = {}

        for header, dimension in zip(dimensionHeaders, row.get('dimensions', [])):
            dict[header] = dimension

        for i, values in enumerate(row.get('metrics', [])):
            for metric, value in zip(metricHeaders, values.get('values')):
                if ',' in value or '.' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = int(value)

        list.append(dict)
    return pd.DataFrame(list)

In [9]:
# GA DATA

GA_response = build('analyticsreporting', 'v4', credentials = CREDENTIALS).reports().batchGet(body = GA_BODY).execute()

(print_response(GA_response)
    .rename(columns = {
        'ga:pagePath': 'PagePath',
        'ga:pageTitle': 'PageTitle'
    })
.to_csv('GA-{}.csv'.format(DATE_RANGES[0]['startDate']), index = False))

In [7]:
# GSC DATA

GSC_response = build('webmasters', 'v3', credentials = CREDENTIALS).searchanalytics().query(siteUrl = URL, body = GSC_BODY).execute()
df_GSC = pd.json_normalize(GSC_response['rows'])

for i, d in enumerate(GSC_BODY['dimensions']):
    df_GSC[d] = df_GSC['keys'].apply(lambda x: x[i])

df_GSC.drop(columns = 'keys', inplace = True)
df_GSC.to_csv('GSC-{}.csv'.format(DATE_RANGES[0]['startDate']), index = False)

In [8]:
df_GA = pd.read_csv('GA-2020-06-30.csv')
df_GSC = pd.read_csv('GSC-2020-06-30.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'GA-2020-06-30.csv'

In [9]:
df_GA

,PagePath,PageTitle,BounceRate,Pageviews,AvgTimeOnPage,AvgSessionDuration,AvgPageLoadTime
0,/tr/primary-unique-foreign-key,"Key, Primary Key, Unique Key ve Foreign Key Ta...",54.429638,7511,507.641135,52.848440,5.228512
1,/tr/pivot-table-nedir,Pivot Table Nedir? Nasıl Kullanılır? | ceaksan,48.717519,6320,663.694960,42.499581,6.867565
2,/tr/primary-unique-foreign-key,"Key, Primary Key, Unique Key ve Foreign Key Ta...",22.555644,5473,666.182952,64.347178,8.077174
3,/tr/join-types-tablo-birlestirme,JOIN Tipleri ve SQL Tablo Birleştirme İşlemler...,47.385554,5244,643.019074,49.546083,5.754970
4,/tr/join-types-tablo-birlestirme,JOIN Tipleri ve SQL Tablo Birleştirme İşlemler...,29.153269,5022,681.349206,55.234298,5.713333
...,...,...,...,...,...,...,...
5792,/wp-json/wp/v2/posts?per_page=100,Page not Found | ceaksan,0.000000,1,6.000000,0.000000,0.000000
5793,/wp-login,Page not Found | ceaksan,0.000000,1,0.000000,0.000000,0.000000
5794,(not set),(not set),0.000000,0,0.000000,2225.000000,0.000000
5795,/l/wp-ads-conversion?wanted=true&offer_code=db...,WP Plugin: Ads Tracking,0.000000,0,0.000000,52.000000,3.127500


In [10]:
df_GSC

,clicks,impressions,ctr,position,query,page
0,2848,24149,0.117934,2.526274,mockup nedir,https://ceaksan.com/tr/mockup-nedir
1,2573,6297,0.408607,1.604891,regex nedir,https://ceaksan.com/tr/regex-regular-expressio...
2,1941,5156,0.376455,1.742630,foreign key nedir,https://ceaksan.com/tr/primary-unique-foreign-key
3,1821,54251,0.033566,4.536893,api nedir,https://ceaksan.com/tr/api-nedir
4,1612,4048,0.398221,1.383152,middleware nedir,https://ceaksan.com/tr/middleware-nedir-ne-ise...
...,...,...,...,...,...,...
9995,1,12,0.083333,8.750000,node js vs express js,https://ceaksan.com/tr/express-js-nedir
9996,1,4,0.250000,7.750000,node js webpack,https://ceaksan.com/tr/posts/webpack-nedir
9997,1,14,0.071429,27.571429,node js xml,https://ceaksan.com/tr/node-js-ile-xml-json-is...
9998,1,4,0.250000,24.500000,node js xml parser,https://ceaksan.com/tr/posts/node-js-ile-xml-j...
